# Converting and updating the main trait database

Last Updated: 2022-04-05  
Quang Nguyen

The role of this script is to take raw tables from Madin et al. and Weissman et al. and merge them. Additionally, include the updated version of the GOLD data set as well. 

We're prepping the latest database for export for evaluation. For this manuscript, we're merging a couple of existing databases:  

1. The comprehensive synthesis of trait-database from [Madin et al. 2020](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7275036/). This database was last updated in 2020. Most of the database's sources are static sources, with the exception of the [GOLD database](https://gold.jgi.doe.gov/downloads). As such, we're merging the existing release of the Madin et al. database with the most recent GOLD release (2022-03-12).  
2. Manual curation of bergey's manual by [Weissman et al.](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-021-04216-2). This database contains manual curation of the Bergey's manual specific to human-associated microbiomes.   

The way we're trying to combine these disparate sources would be to perform something similar to Madin et al. using the [R code](https://github.com/bacteria-archaea-traits/bacteria-archaea-traits/blob/master/R/functions.R) on GitHub. We're going to apply relevant transformations and mappings where apply.  

In [1]:
library(piggyback)
library(data.table)
library(dtplyr)
library(targets)
library(here)
library(stringdist)
library(tidyverse)
here::i_am("analysis/db_prep.ipynb");
setDTthreads(4)

here() starts at /dartfs-hpc/rc/home/k/f00345k/research/microbe_set_trait

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ tidyr::extract()   masks stringdist::extract()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()

here() starts at /dartfs-hpc/rc/home/k/f00345k/research/microbe_set_trait



This code is used to upload data onto GitHub using `piggyback` R package
```r
piggyback::pb_upload(file = here("large_files", "goldData.xlsx"), tag = "0.1", overwrite = TRUE)
```

## Analysis

In [2]:
base <- read_csv(here("data", "condensed_species_NCBI.txt")) %>% 
    select(species_tax_id, superkingdom, phylum, class, order, family, 
           genus, species, metabolism, gram_stain, pathways, 
           carbon_substrates, sporulation, motility, cell_shape) %>% 
    rename("substrate" = carbon_substrates)

Rows: 14893 Columns: 79
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (19): species, genus, family, order, class, phylum, superkingdom, gram_s...
dbl (60): species_tax_id, d1_lo, d1_up, d2_lo, d2_up, doubling_h, genome_siz...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


### GOLD data processing

Let's process GOLD. First, load data from `piggyback` (this is cached). Script follows existing processing pipeline [here](https://github.com/bacteria-archaea-traits/bacteria-archaea-traits/blob/master/R/preparation/gold.R) from Madin et al.  

In [3]:
pth <- here("large_files", "goldData.csv")
if (!file.exists(pth)){
    piggyback::pb_download(file = "goldData.xlsx", dest = here("large_files"), tag = "0.1", repo = "qpmnguyen/microbe_set_trait")
    gold <- readxl::read_xlsx(path = here("large_files", "goldData.xlsx"), sheet = "Organism")
    readr::write_csv(x = gold, file = pth)
}
gold <- read_csv(file = pth);

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 428241 Columns: 42
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (35): ORGANISM GOLD ID, ORGANISM NAME, ORGANISM NCBI SUPERKINGDOM, ORGAN...
dbl  (4): ORGANISM NCBI TAX ID, ORGANISM ISOLATION PUBMED ID, ORGANISM ECOSY...
lgl  (3): ORGANISM SALINITY CONCENTRATION, ORGANISM PRESSURE, ORGANISM CARBO...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


We convert all column names to lower case and replace spaces with `_`. We also rename all columns to be similar to the original base database from Madin et al. Finally, we nest all relevant traits into a column called `traits`

In [4]:
# convert names 
colnames(gold) <- colnames(gold) %>% 
    gsub(x = ., pattern = " ", replacement = "_") %>% 
    tolower() %>% 
    gsub(x = ., pattern = "organism_", replacement = "")


gold_reduced <- gold %>% 
    select(ncbi_tax_id, ncbi_superkingdom,  
            ncbi_phylum, ncbi_class, ncbi_order, ncbi_family, ncbi_genus, ncbi_species, 
            name, gram_stain, metabolism, oxygen_requirement, 
            sporulation, motility, cell_shape) %>% 
    rename("species_tax_id" = ncbi_tax_id,
           "superkingdom" = ncbi_superkingdom,
           "phylum" = ncbi_phylum,
           "class" = ncbi_class,
           "order" = ncbi_order,
           "family" = ncbi_family,
           "genus" = ncbi_genus,
           "species" = ncbi_species,
           "pathways" = metabolism,
           "metabolism" = oxygen_requirement) %>% 
    mutate(metabolism = tolower(metabolism), 
           gram_stain = if_else(gram_stain == "Gram-", "negative", "positive"), 
           sporulation = if_else(sporulation == "Nonsporulating", "no", "yes"), 
           motility = case_when(
               motility == "Nonmotile" ~ "no", 
               motility == "Motile" ~ "yes", 
               TRUE ~ motility
           ), 
           cell_shape = tolower(str_replace(cell_shape,"-shaped","")),
           cell_shape = case_when(
               cell_shape %in% c("rod") ~ "bacillus",
               cell_shape %in% c("sphere", "oval", 
                                 "bean", "coccoid", "ovoid", 
                                 "spore", "Coccus-shaped") ~ "coccus", 
               cell_shape %in% c("helical") ~ "spiral", 
               cell_shape %in% c("curved") ~ "vibrio", 
               cell_shape %in% c("flask", "open-ring", "lancet") ~ "irregular", 
               # only Mycoplasma genitalium for flask 
               # only Thiomicrospira cyclica for open-ring
               # only Nitrolancea hollandica for lancet
               TRUE ~ cell_shape
           )) %>% 
    as.data.table()

In [5]:
# nest traits 
tbl <- gold_reduced %>%
    select(-name) %>%
    group_by(species_tax_id, superkingdom, phylum, class, order, 
             family, genus, species) %>%
    nest(traits = c(gram_stain, pathways, metabolism, 
           cell_shape, motility, sporulation))
    

# a subset of the table that has more than one row per trait nested values 
tbl_munge <- tbl %>% filter(map_lgl(traits, ~{nrow(.x) > 1})) %>% drop_na(species_tax_id)
tbl_munge

Source: local data table [11,177 x 9]
Groups: species_tax_id, superkingdom, phylum, class, order, family, genus, species
Call:
  _DT2 <- `_DT1`[, .(species_tax_id, superkingdom, phylum, class, order, family, genus,
  _DT2 <-   species, gram_stain, pathways, metabolism, sporulation, motility, cell_shape)][
  _DT2 <-   , .(traits = .(.SD)), by = .(species_tax_id, superkingdom, phylum, class,
  _DT2 <-     order, family, genus, species)]
  na.omit(`_DT2`[`_DT2`[, .I[map_lgl(traits, ~{
    nrow(.x) > 1
})], by = .(species_tax_id, superkingdom, phylum, class, order, 
    family, genus, species)]$V1], cols = "species_tax_id")

  species_tax_id superkingdom phylum       class      order family genus species
           <dbl> <chr>        <chr>        <chr>      <chr> <chr>  <chr> <chr>  
1          54005 Bacteria     Firmicutes   Tissierel… Tiss… Pepto… Pept… Pepton…
2           1280 Bacteria     Firmicutes   Bacilli    Baci… Staph… Stap… Staphy…
3         464988 Eukaryota    NA           Cryp

`tbl_munge` is a subset of the GOLD database where more than one row of traits for each species identifier (presumably conflicting information or due to multiple strains within species). Within this, for `gram_stain`, `metabolism`, `sporulation`, `motility`, and `cell_shape`, the trait for the species will be the top trait, assuming that trait is represented more than 50\% of the rows. A lof of the times, the trait vectors are actually identical and we're merely collapsing duplicates. For `pathways` we simply append all the relevant pathways and then return only the uniquely identified ones. 

In [6]:
# This function takes a data frame and a column 
# and selects the response with the highest frequency
select_best <- function(df, column){
    vec <- unlist(df[,..column])
    freq <- as.data.frame(table(vec))
    if (nrow(freq) == 0){
        return(NA_character_)
    } else {
        freq <- freq %>% mutate(prop = Freq/sum(Freq)) %>%
            filter(prop > 0.5) %>% top_n(n = 1, wt = prop)
        return(freq %>% pull(vec) %>% as.vector())
    }
}

# This function then utilizes select_best
# to process entries with duplicates (more than one row)
# for pathways, the goal is to concatenate them
process_duplicates <- function(df){
    # get only unique rows
    df <- unique(df)
    if (nrow(df) == 1){
        return(df)
    }
    v <- c("gram_stain", "pathways", "metabolism", 
           "sporulation", "motility", "cell_shape")
    suppressMessages(res <- map_dfc(v, ~{
        if (.x == "pathways"){
            str_vec <- na.omit(df$pathways) %>% as.vector()
            if (length(str_vec) == 0){
                out <- NA_character_
            } else {
                out <- str_replace(str_vec, pattern = " ", 
                                   replacement = "_") %>% 
                    paste(collapse = ", ")
            }
        } else {
            out <- select_best(df, .x)
        }
        return(out)
    }))
    colnames(res) <- v
    res <- as.data.table(res)
    return(res)
}

In [7]:
tbl_munge_proc <- tbl_munge %>% 
    mutate(traits = map(traits, process_duplicates)) 

head(tbl_munge_proc)

Source: local data table [6 x 9]
Groups: species_tax_id, superkingdom, phylum, class, order, family, genus, species
Call:
  _DT2 <- `_DT1`[, .(species_tax_id, superkingdom, phylum, class, order, family, genus,
  _DT2 <-   species, gram_stain, pathways, metabolism, sporulation, motility, cell_shape)][
  _DT2 <-   , .(traits = .(.SD)), by = .(species_tax_id, superkingdom, phylum, class,
  _DT2 <-     order, family, genus, species)]
  head(na.omit(`_DT2`[`_DT2`[, .I[map_lgl(traits, ~{
    nrow(.x) > 1
})], by = .(species_tax_id, superkingdom, phylum, class, order, 
    family, genus, species)]$V1], cols = "species_tax_id")[, 
    `:=`(traits = map(traits, ..process_duplicates)), by = .(species_tax_id, 
        superkingdom, phylum, class, order, family, genus, species)], 
    n = 6L)

  species_tax_id superkingdom phylum       class      order family genus species
           <dbl> <chr>        <chr>        <chr>      <chr> <chr>  <chr> <chr>  
1          54005 Bacteria     Firmicutes   Ti

We merge by extracting the `species_tax_id` column out of `tbl`, remove all rows with that identifier and then replace that with those from the `tbl_munge` database. We also process some of the trait names themselves and attempt to unify it according to the nomenclature stated in Madin et al. base database. 

In [ ]:
ids <- tbl_munge_proc %>% pull(species_tax_id)

gold_final <- tbl %>% filter(!species_tax_id %in% ids) %>% 
    bind_rows(tbl_munge_proc) %>% 
    as_tibble() %>% 
    unnest(traits) %>% bind_rows(tbl_munge_proc)


head(gold_final)

In [ ]:
# cleaning
# pray not to crash
rm(gold)
rm(gold_reduced)
rm(tbl)
rm(tbl_munge)
gc()

In [ ]:
test_df <- tbl_munge %>% filter(species_tax_id == "1280") %>% as_tibble() %>% pull(traits) %>% .[[1]]

In [ ]:
gold_final %>% tbl

## Processing Weissman et al.  

In [ ]:
weissman <- read_csv(here("data", "weissman.csv"))

# select the relevant columns
weissman <- weissman %>% select(c("taxid_species",
                                  "kingdom", "phylum", "class", "order", "family", "genus", "species",
                                  "Motility_general", "Oxygen.Preference", "Cell.Shape", "Cell.Aggregation",
                                  starts_with("Enzyme.Assays"), 
                                  starts_with("Volatile.Gas.Production"), 
                                  starts_with("Substrate.Utilization")))

Here, we're going to re-format it similar to Madin et al. `Enzyme.Assays` and `Volatile.Gas.Production` is equivalent to `pathways` while `Substrate.Utililization` is equivalent to `carbon_substrates`. Since there might be non-carbon compounds here, we're going to rename Madin et al.'s `carbon_substrates` into just `substrate` here similar to Weissman's database.  

In [ ]:
weissman <- weissman %>% group_by(taxid_species, kingdom, phylum, 
                      class, order, family, genus, species) %>%
    nest(pathways = starts_with(c("Enzyme.Assays", "Volatile.Gas.Production")), 
         substrate = starts_with("Substrate.Utilization")) %>% ungroup() %>% 
    mutate(across(where(is.character), ~na_if(., "0"))) %>% 
    rename("species_tax_id" = taxid_species, "superkingdom" = kingdom, 
           "metabolism" = Oxygen.Preference,
           "motility" = Motility_general, 
           "cell_shape" = Cell.Shape)

# motility, cell shape, metabolism
weissman$cell_shape %>% unique()

weissman <- weissman %>% 
    mutate(motility = if_else(motility == "non-motile", "yes", "no"), 
                    cell_shape = case_when(
                        cell_shape == "rod" ~ "bacillus",
                        cell_shape == "ovoid/coccobacillus" ~ "coccus", 
                        cell_shape == "spirillum/corkscrew" ~ "spiral",
                        TRUE ~ cell_shape
                    ), 
                    metabolism = case_when(
                        metabolism == "microaerophile" ~ "microaerophilic",
                        TRUE ~ metabolism
                    ), 
                    metabolism = str_replace(metabolism, 
                                             pattern = "obe$", replacement = "obic"))

Here we're going to define some functions to process the traits that is internal (collapse into one column) for pathways and for substrates

In [ ]:
#' @param unit A single unit from a list of trait data frames
proc_pathways <- function(unit){
    string <- unit %>% pivot_longer(everything()) %>% 
        filter(value != 0)
    if (nrow(string) == 0){
        return(NA)
    } else {
        string <- string %>% 
            mutate(name = str_replace(name, pattern = "Enzyme.Assays..", replacement = "")) %>%
            mutate(name = str_replace(name, pattern = "Volatile.Gas.Production..", replacement = "synthesis_")) %>%
            mutate(name = str_replace(name, pattern = "\\.$", "")) %>%
            mutate(name = str_replace(name, pattern = "\\.\\.(.*)$", "")) %>%
            mutate(name = str_replace_all(name, pattern = "\\.", "_")) %>% 
            rowwise() %>%
            mutate(name = if_else(str_detect(name, "synthesis_"), 
                                  true = paste(rev(str_split(name, pattern = "_", 
                                                             n = 2)[[1]]), 
                                               collapse = "_"),
                                  false = name)) %>% 
            ungroup() %>%
            pull(name) %>% paste(., collapse = ", ")
    }
        
    return(string)
}

proc_substrate <- function(unit){
    string <- unit %>% pivot_longer(everything()) %>% filter(value != 0)
    if (nrow(string) == 0){
        return(NA)
    } else {
        string <- string %>% mutate(name = str_split(name, pattern = "\\.\\.", 
                                           n = 2, simplify = TRUE)[,2]) %>%
            mutate(name = str_replace(name, "(\\.\\.|\\.)$", "")) %>% 
            mutate(name = str_replace_all(name, "(\\.\\.|\\.)", "_")) %>% 
            pull(name) %>% paste(., collapse = ", ")
    }
    return(string)
}


In [ ]:
substr <- map_chr(weissman$substrate, proc_substrate)
pthway <- map_chr(weissman$pathways, proc_pathways)

weissman <- weissman %>% select(-c(pathways, substrate)) %>% 
  mutate(substrate = substr, pathways = pthway)

## Checking for duplicates

First, we define a function to check for similar sounding names across all the unique pathways and substrates for all the data sets. Here, we use the `stringdist` function from the `stringdist` package. We use the standard OSA metric (also called the Damerau-Levenshtein distance) to query for potential similarly sounding names of identical pathways or compounds. 

In [ ]:
check_matches <- function(df, type=c("pathways","substrates")){
    b_val <- base %>% pull(!!type) %>% unique() %>% str_split(pattern = ", ") %>%
        unlist() %>% unique() %>% na.omit() %>% as.vector()
    
    q_val <- df %>% pull(!!type) %>% unique() %>% str_split(pattern = ", ") %>% 
        unlist() %>% unique() %>% na.omit() %>% as.vector()
    
    check <- map(q_val, ~{
        match <- stringdist(a = .x, b = b_val)
        # match 0 is the same, and match > 2 is too different 
        ret <- b_val[match > 0 & match <= 2]
        if (length(ret) == 0){
            return(NA)
        } else {
            out <- tibble(
                query = rep(.x, length(ret)),
                ref = ret
            )
        }
    })
    check <- check[!sapply(check, function(x) all(is.na(x)))]
    
    return(check)
}

In [ ]:
Reduce(check_matches(weissman, "pathways"), f = rbind)
Reduce(check_matches(weissman, "substrate"), f = rbind)

We can see that for a lot of the compounds the names might be the same but they're actually different. However, there are certain conventions such as "_" for spaces or "-" that we might need to address for the final merge.  

Let's check the GOLD database for similar sounding names. Since GOLD does not have substrate information, we only check for pathways

In [ ]:
Reduce(check_matches(gold_final, "pathways"), f = rbind)

In [ ]:
trim_path <- function(vec){
    vec <- vec %>% tolower() %>% 
        str_split(pattern = "(, |\\|)") %>% 
        map(~{
            str_trim(.x) %>% str_replace_all("\\-", "") %>%
                str_replace_all(" ", "_") %>% unique()
        })
    return(vec)
}


gold_final$pathways <- trim_path(gold_final$pathways)
base$pathways <- trim_path(base$pathways)
weissman$pathways <- trim_path(weissman$pathways)
weissman$substrate <- trim_path(weissman$substrate)
base$substrate <- trim_path(base$substrate)

## Combine all data frames

After munging, let's combine all of the names! The strategy is very similar to handling multiple entries for GOLD. First, we bind all of our databases together. Then, we `group_by` and nest all our trait data into a list. Then we process these lists and return `unique` rows (deduplicated). If the rows are not unique, then we process the non-unique rows by either concatenating the traits together or vote on consensus using the top most represented trait.  

In [ ]:
complete <- bind_rows(
    base %>% mutate(source = "madin"), 
    gold_final %>% mutate(source = "gold"), 
    weissman %>% mutate(source = "weissman")
)

tally <- complete %>% filter(!is.na(species)) %>% 
    group_by(species_tax_id, superkingdom, phylum, class, order, 
             family, genus, species) %>% 
    tally()


multiple_rows <- tally %>% filter(n >= 2) %>% pull(species_tax_id)

reconcile <- complete %>% filter(species_tax_id %in% multiple_rows)

reconcile <- reconcile %>% group_by(species_tax_id, superkingdom, phylum, class, order,
                       family, genus, species, metabolism) %>%
    nest(data = c(metabolism, gram_stain, pathways, substrate, sporulation, motility, cell_shape, cell_aggregation, source)) %>% ungroup()

Let's collapse traits of the same species similar to before and merge everything together to save to a database

In [ ]:
#' @param df This is a data frame of multiple columns, where the columns of 
#'     pathways and substrates are themselves lists 
collapse_trait <- function(df){
    nonlist <- c("metabolism", "gram_stain", "sporulation", "motility", 
                 "cell_shape", "cell_aggregation")    
    out <- suppressMessages(map_dfc(nonlist, ~{
        traits <- df %>% pull(.x)
        traits <- as.data.frame(table(traits))
        if (nrow(traits) >= 1){
            return(traits %>% 
                       mutate(prop = Freq/sum(Freq)) %>%
                       filter(prop == max(prop) & prop >= 0.5) %>% 
                       pull(traits) %>% as.vector()
            )
        } else {
            return(NA_character_)
        }
    }))
    names(out) <- nonlist
    out$pathways <- list(df %>% pull("pathways") %>% 
                             Reduce(f = c, x = .) %>% unique())
    out$substrate <- list(df %>% pull("substrate") %>% 
                              Reduce(f = c, x = .) %>% 
                              unique())
    return(out)
}

reconcile <- reconcile %>% mutate(traits = map(data, collapse_trait)) %>% 
    select(-data)

In [ ]:
complete <- bind_rows(reconcile %>% unnest(traits), 
          complete %>% filter(!species_tax_id %in% multiple_rows))

saveRDS(complete, file = here("output", "db_merged.rds"))
complete %>% mutate(
    pathways = map_chr(pathways, ~{
        paste(.x, collapse = ", ")
    }), 
    substrate = map_chr(substrate, ~{
        paste(.x, collapse = ", ")
    })
) %>% write_csv(x = ., file = here("output", "db_merged.csv"))